In [ ]:
# xmlrpc_server.ipynb

# from SimpleXMLRPCServer import SimpleXMLRPCServer
# from SimpleXMLRPCServer import SimpleXMLRPCRequestHandler

from xmlrpc.server import SimpleXMLRPCServer, SimpleXMLRPCRequestHandler

# import xmlrpclib
from xmlrpc.client import Binary
import datetime
import re
import pandas as pd
import pickle
import numpy as np

from httpx import patch
from jupyter_server.base.handlers import path_regex


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')


def black_list_check_full(surname, name, patronym, birth_date):
    # Проверка формата даты
    date_pattern = r'^\d{2}\.\d{2}\.\d{4}$'
    if not re.match(date_pattern, birth_date):
        return f"Ошибка: Неверный формат даты '{birth_date}'. Ожидается DD.MM.YYYY (например, 22.03.1989)"

    try:
        input_date = datetime.datetime.strptime(birth_date, '%d.%m.%Y')
        current_date = datetime.datetime.now()
        if input_date > current_date:
            return f"Ошибка: Дата рождения '{birth_date}' не может быть позже текущей даты"
    except ValueError:
        return f"Ошибка: Неверная дата '{birth_date}'. Проверьте корректность"

    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf-8')

    try:
        # surname, name, patronym = fio.split(' ')
        surname = surname.title()
        name = name.title()
        patronym = patronym.title()
    except ValueError:
        return f"Ошибка: Неверный формат ФИО. Ожидается 'Фамилия Имя Отчество'"

    frame['Surname'] = frame['Surname'].str.title()
    frame['Name'] = frame['Name'].str.title()
    frame['Patronym'] = frame['Patronym'].str.title()

    mask = (frame['Surname'] == surname) & \
           (frame['Name'] == name) & \
           (frame['Patronym'] == patronym) & \
           (frame['Birth'] == birth_date)

    exist = mask.any()
    if exist:
        return f"{surname} {name} {patronym} ({birth_date}): bad_boy"
    else:
        return f"{surname} {name} {patronym} ({birth_date}): good_boy"

server.register_function(black_list_check_full, 'black_list_check_full')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    channels = img_arr.shape[2] if len(img_arr.shape) > 2 else 1

    for i in range(height):
        for j in range(width):
            if channels in (3, 4):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
                # if channels == 4:
                #     img_arr[i][j][3] = 255 - img_arr[i][j][3]
            else:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

# Бинаризация изображения по порогу (1-255)
def send_back_binarization(bin_data, threshold):
    if not 1 <= threshold <= 255:
        raise ValueError("Порог должен быть в диапазоне 1-255")


    img_arr = pickle.loads(bin_data.data)
    print(f"Сервер: Получено shape={img_arr.shape}, min={img_arr.min()}, max={img_arr.max()}, dtype={img_arr.dtype}")

    if img_arr.max() <= 1.0:
        img_arr = (img_arr * 255).astype(np.uint8)
    else:
        img_arr = (img_arr).astype(np.uint8)

    height = img_arr.shape[0]
    width = img_arr.shape[1]
    channels = img_arr.shape[2] if len(img_arr.shape) > 2 else 1
    print(channels, "channels")
    # binarized_arr = [[[0 for _ in range(channels)] for _ in range(width)] for _ in range(height)]
    binarized_arr = np.zeros_like(img_arr, dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            if channels in (3, 4):
                r = img_arr[i][j][0]
                g = img_arr[i][j][1]
                b = img_arr[i][j][2]
                avg = (r + g + b) // 3
                value = 255 if avg >= threshold else 0
                # print(f"Пиксель [{i},{j}]: value={value} and avg={avg}")
                for c in range(min(3, channels)):
                    binarized_arr[i][j][c] = value
                if channels == 4:
                    binarized_arr[i][j][3] = img_arr[i][j][3]
            elif channels == 1:
                value = 255 if img_arr[i][j][0] >= threshold else 0
                # print(f"Пиксель [{i},{j}]: value={value} and avg={avg}")
                binarized_arr[i][j][0] = value

    binarized_arr = np.array(binarized_arr, dtype=np.uint8)
    print(f"Сервер: После бинаризации min={np.min(binarized_arr)}, max={np.max(binarized_arr)}")
    pimg = pickle.dumps(binarized_arr)
    return Binary(pimg)
    # return binarized_arr
server.register_function(send_back_binarization, 'send_back_binarization')


# Разворот изображения относительно вертикали
def send_back_flip_vertical(bin_data):
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    channels = img_arr.shape[2] if len(img_arr.shape) > 2 else 1
    # flipped_arr = [[[0 for _ in range(channels)] for _ in range(width)] for _ in range(height)]

    for i in range(height):
        for j in range(width // 2):
            for c in range(channels):
                img_arr[i][j][c], img_arr[i][width - 1 - j][c] = img_arr[i][width - 1 - j][c], img_arr[i][j][c]
                # flipped_arr[i][width - 1 - j][c] = img_arr[i][j][c]

    pimg = pickle.dumps(img_arr)
    return Binary(pimg)
server.register_function(send_back_flip_vertical, 'send_back_flip_vertical')


print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...
